## Input data requirements

The input ice sheet model should be saved as a netCDF file. There are several requirements for the comparison:

### `Lithk` variable

The uploaded model to contain thickness data (the `lithk` variable) for the comparison.

### Rectangular grid

At time of writing, models *must* be defined on a rectangular X-Y grid in the ISMIP6 standard projected polar-stereographic space. (Note, NOT a lat-lon grid!) The ISMIP6 standard projection is defined [here](http://www.climate-cryosphere.org/wiki/index.php?title=ISMIP6_wiki_page). 


In [1]:
# set geospatial python3 kernel to provide needed packages
import os,sys
import glob as glob
import numpy as np
import h5py
import xarray as xr
import glob as glob
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import Point



import cartopy
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader

# note: suppress numpy.dtype size changed warnings
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")

warnings.filterwarnings('ignore')

### Set ice sheet region and start and end date

In [2]:
# Define the flag for the ice sheet region Greenland or Antarctica
icesheet = "Antarctica" # Change to "Antarctica" Or "Greenland"

# Define start and end dates
start_date = np.datetime64('2000-01-01')
end_date = np.datetime64('2015-01-01')

### Select  variable for mass balance comparision

In [3]:
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import VBox, Dropdown

def select_variable():
    # User Input Widgets
    variable_w = Dropdown(options=["Mass balance (Gt/yr)", "Dynamics mass balance anomaly (Gt/yr)"], value='Mass balance (Gt/yr)', description='variable:')
    
    # Collect all widgets in a vertical layout
    all_widgets = VBox([variable_w])
    display(all_widgets)
    
    # Return the widget itself for interaction
    return variable_w

# Display the dropdown and capture user selection
variable_widget =select_variable()

# Observe and set the projection based on user input
def handle_variable_change(change):
    var1 = change['new']

# Attach an observer to the widget to update when selection changes
variable_widget.observe(handle_variable_change, 'value')

In [4]:
# Selected variable
mass_balance_column=variable_widget.value
print(mass_balance_column)

Mass balance (Gt/yr)


### Select projection, shapefile and csv file for imbie

In [5]:
# Set shapefile path and projection and imbie csv_filebased on the ice sheet region
if icesheet == "Greenland":
    projection = "EPSG:3413"  # Greenland
    #Set the model data dir path
    nc_filename='/home/jovyan/CmCt/notebooks/Gravimetry/lithk_GIS_JPL_ISSM_asmb.nc'
    shapefile_path = "/home/jovyan/CmCt/notebooks/IMBIE/Greenland_Basins_PS_v1.4.2/Greenland_Basins_PS_v1.4.2.shp"
    csv_file_path = '/home/jovyan/CmCt/notebooks/IMBIE/imbie_greenland_2022_Gt_partitioned_v0.csv'
    # csv_file_path = '/home/jovyan/CmCt/notebooks/IMBIE/imbie_greenland_2021_Gt.csv'
    
elif icesheet== "Antarctica":
    projection = "EPSG:3031"  # Antarctica
    #Set the model data dir path
    nc_filename='/home/jovyan/CmCt/notebooks/Gravimetry/lithk_AIS_AWI_PISM1_hist_std.nc'
    shapefile_path = "/home/jovyan/CmCt/notebooks/IMBIE/ANT_Basins_IMBIE2_v1.6/ANT_Basins_IMBIE2_v1.6.shp"
    # csv_file_path = '/home/jovyan/CmCt/notebooks/IMBIE/imbie_antarctica_2021_Gt.csv'
    csv_file_path = '/home/jovyan/CmCt/notebooks/IMBIE/imbie_antarctica_2022_Gt_partitioned_v0.csv'
else:
    raise ValueError("Invalid iceshee value. Must be 'Greenland' or 'Antarctica'.")


### Caluclate  model mass balance for each basin and total mass cbalance for whole region

In [6]:
# Load the dataset(model data)
gis_ds = xr.open_dataset(nc_filename)
lithk = gis_ds['lithk']

if icesheet == "Greenland":
    # Interpolate lithk values at the start and end dates
    lithk_start = lithk.interp(time=start_date).data.transpose().flatten()
    lithk_end = lithk.interp(time=end_date).data.transpose().flatten()

elif icesheet== "Antarctica":
    import cftime 
    # Convert the start and end dates to cftime.datetime objects
    start_date1 = cftime.DatetimeNoLeap(2006, 1, 1)
    end_date1 = cftime.DatetimeNoLeap(2015, 1, 1)
    
    # Interpolate lithk values at the start and end dates using the nearest method
    lithk_start = lithk.sel(time=start_date1, method='nearest').data
    lithk_end = lithk.sel(time=end_date1, method='nearest').data



# Calculate the difference
lithk_delta = lithk_end - lithk_start

# Replace NaN values with 0
lithk_delta[np.isnan(lithk_delta)] = 0


# Change Ice thickness unit from (m) to mass (kg) to gigatonnes(Gt)
# ice thickness*area* density of ice* 1e-12

rho_ice = 934 # (density of ice, kg/m^3)

#calculate area = x_resolution*y_resolution
x_coords = gis_ds['x'].values
y_coords = gis_ds['y'].values
x_resolution = abs(x_coords[1] - x_coords[0])
y_resolution = abs(y_coords[1] - y_coords[0])

lithk_delta = (lithk_delta * x_resolution*y_resolution)*rho_ice * 1e-12


# Create a list of Point geometries from coordinate grids
points = [Point(x, y) for x in x_coords for y in y_coords]

# Flatten lithk_delta to match the points list 
lithk_delta_flat = lithk_delta.flatten()

# Create DataFrame
lithk_df = pd.DataFrame({
    'geometry': points,
    'lithk_delta': lithk_delta_flat
})

# Convert DataFrame to GeoDataFrame
lithk_gdf = gpd.GeoDataFrame(lithk_df, geometry='geometry', crs=projection)


# Load basin shapefile 
basins_gdf = gpd.read_file(shapefile_path)


# Perform spatial join
# joined_gdf = gpd.sjoin(lithk_gdf, basins_gdf, how="inner", op='intersects')
joined_gdf = gpd.sjoin(lithk_gdf, basins_gdf, how="inner", predicate='intersects')


# Sum lithk_delta values by basin
basin_mass_change_sums = joined_gdf.groupby('index_right')['lithk_delta'].sum()
print('Basin_mass_change_sums =', basin_mass_change_sums)

if icesheet == "Antarctica":
    # Sum lithk_delta values by the 'Regions' column
    region_mass_change_sums = joined_gdf.groupby('Regions')['lithk_delta'].sum()
    print('Region_mass_change_sums =', region_mass_change_sums)


# Sum all of the basin mass change
model_total_mass_balance= basin_mass_change_sums.sum()
print('Total_mass_change_sum =', model_total_mass_balance)


Basin_mass_change_sums = index_right
0      -4.885852
1       0.011065
2     -11.727497
3     -71.635292
4      22.095102
5      28.346788
6     -31.312515
7      -8.573805
8     -21.483847
9     -37.235355
10    -82.697937
11    -13.701855
12     36.779713
13     -4.665155
14   -132.672729
15     -5.731662
16     -1.479897
17     -8.639709
18    -54.555046
Name: lithk_delta, dtype: float32
Region_mass_change_sums = Regions
East        -146.848801
Islands       -4.885852
Peninsula    -15.851269
West        -236.179550
Name: lithk_delta, dtype: float32
Total_mass_change_sum = -403.76547


### Imbie mass balance

In [7]:
def sum_MassBalance(csv_file_path):
    # Load the CSV file
    mass_balance_data = pd.read_csv(csv_file_path)
    
    # Adjust the column names based on the CSV content
    date_column = 'Year'
    # mass_balance_column = 'Mass balance (Gt/yr)'
    
    # Convert the 'Year' column to datetime format
    mass_balance_data[date_column] = pd.to_datetime(mass_balance_data[date_column], format='%Y')
    
    # Filter the data between the start and end dates
    filtered_data = mass_balance_data[(mass_balance_data[date_column] >= start_date) & 
                                      (mass_balance_data[date_column] <= end_date)].copy()
    
    # Convert Mass balance from Gt/yr to Gt/month
    filtered_data.loc[:, 'Mass_balance_Gt'] = filtered_data[mass_balance_column] / 12
    
    # Sum the total mass balance in the filtered range
    imbie_total_mass_change_sum = filtered_data['Mass_balance_Gt'].sum()
    
    # # Print the results
    # print(f"Total {mass_balance_column} from {start_date} to {end_date}: {imbie_total_mass_change_sum} Gt")

    return imbie_total_mass_change_sum

### Calculate mass balance difference of imbie and model data

In [8]:
#Total mass balance
imbie_total_mass_change_sum=sum_MassBalance(csv_file_path)
# Print the results
print(f"Total {mass_balance_column} from {start_date} to {end_date}: {imbie_total_mass_change_sum} Gt")


#Difference of imbie-model  mass change 
delta_masschange=imbie_total_mass_change_sum-model_total_mass_balance
print(f"Difference of imbie-model  mass change ({mass_balance_column}): {delta_masschange}")

Total Mass balance (Gt/yr) from 2000-01-01 to 2015-01-01: -2496.2130333333334 Gt
Difference of imbie-model  mass change (Mass balance (Gt/yr)): -2092.447560921224


In [9]:
if icesheet == "Antarctica":
    #Region total mass balance
    csv_file_path_east = '/home/jovyan/CmCt/notebooks/IMBIE/imbie_east_antarctica_2022_Gt_partitioned_v0.csv'
    csv_file_path_west = '/home/jovyan/CmCt/notebooks/IMBIE/imbie_west_antarctica_2022_Gt_partitioned_v0.csv'
    csv_file_path_peninsula= '/home/jovyan/CmCt/notebooks/IMBIE/imbie_antarctic_peninsula_2022_Gt_partitioned_v0.csv'
    
    # Calculate total mass for each region
    imbie_total_mass_change_sum_east=sum_MassBalance(csv_file_path_east)
    imbie_total_mass_change_sum_west=sum_MassBalance(csv_file_path_west)
    imbie_total_mass_change_sum_peninsula=sum_MassBalance(csv_file_path_peninsula)
    

    #Difference of imbie-model  mass change  for each region
    delta_masschange_east=imbie_total_mass_change_sum_east-region_mass_change_sums['East']    
    delta_masschange_west=imbie_total_mass_change_sum_west-region_mass_change_sums['West']   
    delta_masschange_peninsula=imbie_total_mass_change_sum_peninsula-region_mass_change_sums['Peninsula']



    # Print the results
    print(f"Total {mass_balance_column}  for east region from {start_date} to {end_date}: {imbie_total_mass_change_sum_east} Gt")
    print(f"Difference of imbie-model  mass change  fro east region ({mass_balance_column}): {delta_masschange_east}")
    
    print(f"Total {mass_balance_column}  for west region from {start_date} to {end_date}: {imbie_total_mass_change_sum_west} Gt")
    print(f"Difference of imbie-model  mass change  for west region ({mass_balance_column}): {delta_masschange_west}")
    
    print(f"Total {mass_balance_column}  for peninsula region from {start_date} to {end_date}: {imbie_total_mass_change_sum_peninsula} Gt")
    print(f"Difference of imbie-model  mass change for peninsula region ({mass_balance_column}): {delta_masschange_peninsula}")    
    
    

Total Mass balance (Gt/yr)  for east region from 2000-01-01 to 2015-01-01: 117.9607583333333 Gt
Difference of imbie-model  mass change  fro east region (Mass balance (Gt/yr)): 264.80955899251296
Total Mass balance (Gt/yr)  for west region from 2000-01-01 to 2015-01-01: -2056.953383333333 Gt
Difference of imbie-model  mass change  for west region (Mass balance (Gt/yr)): -1820.7738331624346
Total Mass balance (Gt/yr)  for peninsula region from 2000-01-01 to 2015-01-01: -557.2204 Gt
Difference of imbie-model  mass change for peninsula region (Mass balance (Gt/yr)): -541.3691312316895
